# CS229, Fall 2017
## Problem Set 4: EM, DL & RL

This is my solutions for CS229 - Fall 2017: Machine Learning taught by Andrew Ng.

The material for Problem Set 4 is here: [ps4](https://github.com/nmduonggg/ML-CS229/blob/master/Problem%20Set%204/ps4.pdf)

This notebook contains the solution for __Question 4: Independent components analysis__

In [1]:
#!pip install pyaudio sounddevice

### Question 4)

The gradient descent apply for ICA with the assumption that data source follows sigmoid distribution

$$
W = W + \alpha \bigg(
    \begin{bmatrix}
        1 - 2g(w_1^T x^i \\
        1 - 2g(w_2^T x^i) \\
        ... \\
        1 - 2g(w_d^T x^i)
    \end{bmatrix}
\bigg) {x^i}^T + (W^T)^{-1}
$$

where:

- $N$: Number of mixture observations
- $d$: Number of (mixed) signals

In [2]:
import sounddevice as sd
import numpy as np

Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('data/mix.dat')
    return mix

def play(vec):
    sd.play(vec, Fs, blocking=True)


In [3]:
def sigmoid(x):
    return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))

In [41]:
def unmixer(X):
    M, N = X.shape
    W = np.eye(N)

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    
    for alpha in anneal:
        for x in X:
            sm = sigmoid(np.dot(W, x.T))
            gradW = np.outer(1 - 2*sm, x) + np.linalg.inv(W.T) # if we use np.dot -> might be 1D scalar
            W += alpha*gradW
        
    return W

In [45]:
def unmix(X, W):
    S = np.zeros(X.shape)
    S = X.dot(W.T)
    return S

In [43]:
# Play mixed signal
X = normalize(load_data())

for i in range(X.shape[1]):
    print('Playing mixed track %d' % i)
    play(X[:, i])

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4


In [46]:
W = unmixer(X)
S = normalize(unmix(X, W))

for i in range(S.shape[1]):
    print('Playing separated track %d' % i)
    play(S[:, i])

Separating tracks ...
Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
